<a href="https://colab.research.google.com/github/johyunkang/MLwithPythonCookbook/blob/main/10_%ED%8A%B9%EC%84%B1%EC%84%A0%ED%83%9D%EC%9D%84%EC%82%AC%EC%9A%A9%ED%95%9C_%EC%B0%A8%EC%9B%90%EC%B6%95%EC%86%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 10.0 소개

9장에서 새로운 특성을 만드는 식으로 특성 행렬의 차원을 축소하는 방법을 설명했음

이런 특성은 (이상적으로) 훨씬 적은 차원으로 좋은 품질의 모델을 동일하게 훈련할 수 있음.

이를 `특성 추출(feature extraction)`이라고 부름

이 장에서 또 다른 접근 방법으로 고품질의 정보가 많은 특성은 선택하고 덜유용한 특성은 버리는 방식을 다루겠음

이를 `특성 선택(feature selection)`이라고 부름

**특성 선택** 방식에는 필터(`filter`), 래퍼(`wrapper`), 임베디드(`embeded`) 세 가지가 있음
- 필터 방식은 통계적인 속성을 조사하여 가장 뛰어난 특성을 선택
- 래퍼 방식은 시행착오를 통해 가장 높은 품질의 예측을 만드는 특성의 부분 조합을 찾음
- 임베디드 방식은 학습 알고리즘의 훈련 단계를 확장하거나 일부로 구성하여 가장 좋은 특성의 부분 조합을 선택

## 10.1 분산을 기준으로 수치 특성 선택하기

수치형 특성 중에서 분산이 낮은 특성 (즉, 정보가 거의 없는 특성)을 삭제하고 싶음

In [1]:
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold

iris = datasets.load_iris()

feature = iris.data
target = iris.target

print('feature shape:', feature.shape, ', target shape:', target.shape)
print('feature sample:', feature[:3])
print('target sample:', target[:3])

feature shape: (150, 4) , target shape: (150,)
feature sample: [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]]
target sample: [0 0 0]


In [2]:
# 기준값 만들기
threshold = VarianceThreshold(threshold=0.5)

# 기준값 보다 높은 특성을 선택
features_high_variance = threshold.fit_transform(feature)

# 선택한 특성 확인
print('선택한 특성 shape:', features_high_variance.shape)
print('선택한 특성 확인:', features_high_variance[:3])

선택한 특성 shape: (150, 3)
선택한 특성 확인: [[5.1 1.4 0.2]
 [4.9 1.4 0.2]
 [4.7 1.3 0.2]]


`분산기준설정(variance thresholding; VT)`를 사용할 때는 두 가지를 기억해야 함
- 분산은 원점에 맞춰진 값이 아님. 즉 특성의 제곱 단위임. 따라서 특성의 단위가 서로 다르면 VT가 작동하지 않음
- 분산의 기준값을 수동으로 선택하기 때문에 어떤 값이 좋은지 판단할 수 있어야 함.

In [3]:
print('분산을 확인합니다:', threshold.variances_)

분산을 확인합니다: [0.68112222 0.18871289 3.09550267 0.57713289]


- 특성이 (평균이 0이고 단위 분산으로) 표준화되어 있으면 당연하지만 분산 기준 선택 방식은 올바르게 작동하지 않습니다.

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
feature_std = scaler.fit_transform(feature)

# 각 특성의 분산을 계산
selector = VarianceThreshold()
print('표준화된 분산 확인 시...:', selector.fit(feature_std).variances_)

표준화된 분산 확인 시...: [1. 1. 1. 1.]


## 10.2 분산을 기준으로 이진 특성 선택하기 

**이진 범주형 특성(binary categorical feature)**에서 분산이 낮은 특성(적은 정보를 가진 특성)을 삭제하고 싶음

`베르누이 확률 변수(Bernoulli random variable)`의 분산이 기준값 이상인 특성을 선택

In [7]:
from sklearn.feature_selection import VarianceThreshold

# 예제 특성 행렬 생성
# 특성 0 : 80%가 클래스 0
# 특성 1 : 80%가 클래스 1
# 특성 2 : 60%가 클래스 0, 40%는 클래스 1
features = [[0, 1, 0],
            [0, 1, 1],
            [0, 1, 0],
            [0, 1, 1],
            [1, 0, 0]]

# 분산을 기준으로 선택
thresholder = VarianceThreshold(threshold=(0.75 * (1 - 0.75)))
print(thresholder.fit_transform(features))
print('속성의 분산 확인:', thresholder.variances_)

[[0]
 [1]
 [0]
 [1]
 [0]]
속성의 분산 확인: [0.16 0.16 0.24]


이진특성 (베르누이 확률변수)의 분산은 다음과 같이 계산함

$Var(x) = p(1-p)$

위에서 p는 클래스 1의 샘플 비율

In [8]:
# 아래 코드는 numpy var 를 이용하여 위에서 구한 분산을 동일하게 계산
import numpy as np
np.var(features, axis=0)

array([0.16, 0.16, 0.24])

이진 특성에 `var` 함수를 사용하는 것은 이진 특성일 때 베르누이 확률 변수의 분산과 같기 때문임.

분산 공식을 사용해 간단히 유도해볼 수 있음

분산식을 다음과 같이 풀어 쓸 수 있음

$Var(x) = \dfrac{1}{n} \displaystyle \sum_{i=1}^n{(x_i - \mu)^2} = \dfrac{1}{n}\left(\sum_{i=1}^nx_i^2 - 2\mu\sum_{i=1}^nx_i + n\mu^2 \right)$

0, 1로 이루어진 이진 특성일 경우 x<sub>i</sub><sup>2</sup>은 x<sub>i</sub>와 같으므로 1/n 을 곱하면 첫 번째 항은 평균과 같아집니다. 

두 번째 항도 마찬가지로 1/n을 곱하면 평균의 제곱으로 표현할 수 있습니다. 결국 다음과 같이 정리됩니다.

$= \dfrac {1}{n} \displaystyle \sum_{i=1}^nx_i - 2 \mu \dfrac{1}{n} \sum_{i=1}^nx_i + \mu^2 = \mu - 2 \mu^2 + \mu^2 = \mu - \mu^2 = \mu (1 - \mu )$

이진 특성의 평균 뮤는 클래스 1의 샘플 비율과 같습니다. 따라서 `var` 함수로 이진 특성의 분산을 계산하면 **베르누이 확률 변수**의 분산 **p(1-p)**와 같습니다.

`threshold` 매개변수의 기본값은 0으로 모든 특성을 선택합니다.

## 10.3 상관관계가 큰 특성 다루기

과제 : 특성 행렬에서 일부 특성의 상관관계가 크다고 의심됩니다.

In [16]:
import pandas as pd
import numpy as np

# 상관관계가 큰 두 개의 특성을 가진 특성 행렬을 생성
features = np.array( [ [1, 1, 1],
                       [2, 2, 0],
                       [3, 3, 1],
                       [4, 4, 0],
                       [5, 5, 1],
                       [6, 6, 0],
                       [7, 7, 1],
                       [8, 7, 0],
                       [9, 7, 1] ] )

df = pd.DataFrame(features)
display(df.head())
print('\n\n')

# 상관행렬 생성 (절대값)
corr_matrix = df.corr().abs()
print('절대값 상관관계 행렬')
display(corr_matrix)

print('\n\n행렬의 상삼각(upper trangle) 행렬을 선택')
upper = corr_matrix.where( np.triu( np.ones(corr_matrix.shape),
                                  k=1).astype(bool))
print('type:', type(upper))
display(upper)

# 상관 계수가 0.95 보다 큰 특성 열의 인덱스를 찾음
to_drop = [ column for column in upper.columns if any( upper[column] > 0.95 ) ]

# 특성 삭제
df.drop(df.columns[to_drop], axis=1).head(3)

,0,1,2
0,1,1,1
1,2,2,0
2,3,3,1
3,4,4,0
4,5,5,1





절대값 상관관계 행렬


,0,1,2
0,1.000000,0.976103,0.000000
1,0.976103,1.000000,0.034503
2,0.000000,0.034503,1.000000




행렬의 상삼각(upper trangle) 행렬을 선택
type: <class 'pandas.core.frame.DataFrame'>


,0,1,2
0,NaN,0.976103,0.000000
1,NaN,NaN,0.034503
2,NaN,NaN,NaN


,0,2
0,1,1
1,2,0
2,3,1


상관관계 행렬은 `numpy`의 `corrcoef` 함수로 구할 수도 있습니다.

이 함수는 특성이 `행`에 놓여 있을 것으로 기대함.

특성이 열에 놓여 있다고 알려주려면 `rowvar` 매개변수를 `False`로 지정

In [17]:
np.corrcoef(features, rowvar=False)

array([[ 1.        ,  0.97610336,  0.        ],
       [ 0.97610336,  1.        , -0.03450328],
       [ 0.        , -0.03450328,  1.        ]])

## 10.4 분류 작업에 관련 없는 특성 삭제하기

과제 : 범주형 타깃 벡터에서 관련 없는 특성을 삭제하고 싶습니다.


해결 : 범주형 특성이라면 각 특성과 타겟 벡터 사이의 카이제곱 통계를 계산

In [25]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

# 데이터 로드
iris = load_iris()
features = iris.data
target = iris.target
print('features raw:', features[:3])
# 범주형 데이터를 정수형으로 변환
features = features.astype(int)
print('\n\nfeatures int 변환:', features[:3])

# 카이제곱 통계값이 가장 큰 특성 두 개를 선택
chi2_selector = SelectKBest(chi2, k=2)
features_kbest = chi2_selector.fit_transform(features, target)

# 결과 확인
print('\n\n원본 특성 개수 :', features.shape)
print('줄어든 특성 개수 :', features_kbest.shape)

features raw: [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]]


features int 변환: [[5 3 1 0]
 [4 3 1 0]
 [4 3 1 0]]


원본 특성 개수 : (150, 4)
줄어든 특성 개수 : (150, 2)


특성이 수치형(quantitative)이면 각 특성과 벡터 사이에서 분산분석(ANOVA)의 `F-값`을 계산합니다.

In [28]:
# F값이 가장
fvalue_selector = SelectKBest(f_classif, k=2)
features_kbest = fvalue_selector.fit_transform(features, target)

print('\n\n원본 특성개수:', features.shape)
print('F-value를 이용한 줄어든 특성 개수:', features_kbest.shape)



원본 특성개수: (150, 4)
F-value를 이용한 줄어든 특성 개수: (150, 2)
